# Machine Learning Project

Following on from the Principles of Data Science Project in part one, this project will use the information gain to explore and prepare the AutoTrader car listings dataset for the purpose of machine learning models.

Initial steps are to load the libraries needed for the purpose of data processing, exploration and machine learning. The libraries used in this project are:

**Data Processing**
- Numpy
- Pandas
- Scipy
- Math

**Data Visualisation**
- Matplotlib
- Seaborn
- PrettyTable

**Machine Learning**
- SKlearn
- Category Encoders
- Light GBM
- XGBoost
- SHAP
- PPScore


In [ ]:
# Import data libraries
import numpy as np
import pandas as pd
from scipy import stats
import scipy
import math

# Display float type to 5 decimal places, no scientific format
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Visualisation libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Configure plotting backend
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(
    rc={ "figure.figsize": (12,8) },
    style="ticks", context="notebook", font_scale=1.2
)
sns.color_palette('Blues', as_cmap=True);

# Machine Learning libraries
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from category_encoders.target_encoder import TargetEncoder

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
import lightgbm as lgb
import xgboost as xgb

import shap
import ppscore as pps

from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import GridSearchCV


# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

print('Libraries Imported!')

In [ ]:
# Read the data into dataframe, print first few rows.
cars = pd.read_csv('adverts.csv', index_col=['public_reference'])
cars.head()

In [ ]:
# Drop any duplicate observations from the dataset
cars = cars.drop_duplicates()

## Meaning and Type of Features

Look at the features in the dataset and evaluate meaning.

In [ ]:
# Print the number of rows and columns in the dataset
print('The shape of the dataset is: {}'.format(cars.shape))
print('')

# Get feature information and data types
cars.info();

In [ ]:
# Change feature names so they are easier to access
cars = cars.rename(columns={
    'reg_code':'reg',
    'standard_colour':'colour',
    'standard_make':'make',
    'standard_model':'model',
    'vehicle_condition':'condition',
    'year_of_registration':'year',
    'body_type':'type',
    'crossover_car_and_van':'car_van',
    'fuel_type':'fuel'})

In [ ]:
# Show first five rows from the dataset
cars.head()

## Analysis of Distributions - Numeric

Visualise distribution of numeric features.

In [ ]:
features = ['mileage','year','price']

In [ ]:
# Summary statistics for numerical features in the dataset.
cars[features].describe()

In [ ]:
# Define function for calculating skewness and kurtosis
def get_skew(df, col):
    print('{} Skewness {}: '.format(col, df[col].skew()))
    print('{} Kurtosis {}: '.format(col, df[col].kurt()))

In [ ]:
# Create figure for subplots
fig, ax = plt.subplots(2,2, figsize=(12,8), constrained_layout=True)

# View boxplot distribution for mileage
sns.boxplot(x='mileage', data=cars, ax=ax[0,0]);
ax[0,0].set_title('Box Plot Distribution of Mileage (A)');

# View histplot distribution for mileage
sns.boxplot(x='mileage', data=cars, ax=ax[0,1], showfliers=False);
ax[0,1].set_title('Box Plot Distribution of Mileage without Outliers (B)');


# View histogram distribution for mileage
sns.histplot(x='mileage', data=cars, ax=ax[1,0], bins=30);
ax[1,0].set_title('Histogram Distribution of mileage (C)');

# View histplot distribution for subset of cars with mileage less than 200k,
# split by condition
sns.histplot(x='mileage', data=cars.loc[cars['mileage'] <= 125000], bins=40, hue='condition', multiple='stack', ax=ax[1,1]);
ax[1,1].set_title('Histogram Distribution Without Outliers (D)');

plt.savefig('figure/figure1.png')

In [ ]:
# Get skewness and kurtosis of mileage feature
get_skew(cars, 'mileage')

In [ ]:
# Create figure for subplots
fig, ax = plt.subplots(2,2, figsize=(12,8), constrained_layout=True)

# View boxplot distribution for year
sns.boxplot(x='year', data=cars, ax=ax[0,0]);
ax[0,0].set_title('Box Plot Distribution of Year (A)');

# View boxplot distribution without outliers
sns.boxplot(x='year', data=cars, showfliers=False, ax=ax[0,1]);
ax[0,1].set_title('Box Plot Distribution of Year Without Outliers (B)');

# View histogram distribution for year
sns.histplot(x='year', data=cars, ax=ax[1,0], bins=30);
ax[1,0].set_title('Histogram Distribution of year (C)');

# View histogram distribution for years above 2000 split by condition
sns.histplot(x='year', data=cars.loc[cars['year'] >= 2006], bins=15, hue='condition', multiple='stack', ax=ax[1,1]);
ax[1,1].set_title('Histogram Distribution Without Visual Outliers (D)');

In [ ]:
# Get skewness and kurtosis of year feature
get_skew(cars, 'year')

In [ ]:
# Create figure for subplots
fig, ax = plt.subplots(2,2, figsize=(12,8), constrained_layout=True)

# View boxplot distribution for price
sns.boxplot(x='price', data=cars, ax=ax[0,0]);
ax[0,0].set_title('Box Plot Distribution of Price (A)');

sns.boxplot(x='price', data=cars, ax=ax[0,1], showfliers=False);
ax[0,1].set_title('Box Plot Distribution without Outliers (B)')


# View histogram distribution for price
sns.histplot(x='price', data=cars, ax=ax[1,0], bins=30);
ax[1,0].set_title('Histogram Distribution of price (C)');

# View histogram distribution for price under 150k split by condition
sns.histplot(x='price', data=cars.loc[cars['price'] <= 40000], bins=21, kde=True, ax=ax[1,1]);
ax[1,1].set_title('Histogram Distribution of price Without Outliers (D)');

plt.savefig('figure/figure3.png')

In [ ]:
# Get skewness and kurtosis of price feature
get_skew(cars, 'price')

## Analysis of Distributions - Categoric

In [ ]:
# List of categorical features in the dataset
features = ['colour', 'make', 'model', 'condition', 'type', 'car_van', 'fuel']

# Descriptive statistics of categorical features
cars[features].describe(include='all')

In [ ]:
# Loop through categorical features and plot frequency distribution of values
for feature in features:
    print(feature)
    print(cars[feature].value_counts(normalize=True))
    print('') 

In [ ]:
# Create figure for subplots
fig, axs = plt.subplots(4, 2, figsize=(12,20), constrained_layout=True)

# Loop through categorical features and plot frequency distribution of values
for feature, ax in zip(features, axs.ravel()):
    cars[feature].value_counts().head(20).plot(kind='bar', ax=ax);
    ax.set_title('Counts Distribution of {} Descending'.format(feature))
   
# Delete final plot as only 7 necessary.
fig.delaxes(axs[3,1])

plt.savefig('figure/figure4.png')

## Analysis of Predictive Power of Features

Which features are the best for predicting price?

In [ ]:
fig = plt.figure(figsize=(6,6))
sns.heatmap(cars.corr(), annot=True, cmap='Blues');

plt.show()

In [ ]:
for feature in cars.columns:
    print(cars.groupby(feature)['price'].median().sort_values(ascending=False))

In [ ]:
pps.predictors(cars, 'price').sort_values(by='model_score')

## Dealing with Missing Values and Noise

In [ ]:
# Get number of missing values for each column
cars.isnull().sum()

In [ ]:
# Use these values to replace the values in the fuel column.
cars['fuel'] = cars.groupby(['make'])['fuel'].transform(lambda x: x.fillna(x.mode(dropna=False).iloc[0]))

In [ ]:
# Replace null values with the grouped mode of 'make' and 'model'
cars['type'] = cars.groupby('make')['type'].transform(lambda x: x.fillna(x.mode(dropna=False).iloc[0]))

In [ ]:
# Replace with mode of full dataset
cars['type'] = cars['type'].fillna(cars['type'].mode()[0])

# Check for missing values in type
cars['type'].isnull().sum()

In [ ]:
# Set 'year' and 'reg' to 2021 for 'NEW' cars
cars.loc[cars['condition'] == 'NEW', 'year'] = 2021
cars.loc[cars['condition'] == 'NEW', 'reg'] = 21

# Check missing values
cars[['year','reg']].isnull().sum()

In [ ]:
# Fill the missing values with the median of the feature
cars['year'] = cars['year'].fillna(cars['year'].median())

In [ ]:
# Group years into bins
year_labels = ['Very Old', 'Old', 'Neutral Age', 'New', 'Very New']
cars['year_bins'] = pd.qcut(cars['year'], q=[0, 0.2, 0.4, 0.6, 0.8, 1], labels=year_labels)

In [ ]:
# Check skewness of mileage distribution of each year bin.
for bins in year_labels:
    print('Year bin = {}'.format(bins))
    get_skew(cars.loc[cars['year_bins'] == bins], 'mileage')
    print('')

In [ ]:
# Check mean and median values of mileage for each year_bin.
cars.groupby('year_bins')['mileage'].agg(['mean', 'median'])

In [ ]:
# Replace null values with the grouped mileage median of 'year_bins'
cars['mileage'] = cars.groupby('year_bins')['mileage'].transform(lambda x: x.fillna(x.median()))

In [ ]:
# Check for missing values
cars['colour'].isnull().sum()

In [ ]:
# Keep missing values in colour as 'Undefined'
cars['colour'] = cars['colour'].fillna('Undefined')

### Outliers

In [ ]:
lower, upper = np.percentile(cars['price'], [0.01, 99.99])

In [ ]:
cars.loc[cars['price'] < lower].head(10)

In [ ]:
cars.loc[cars['price'] > upper]

In [ ]:
def remove_outliers(df,columns):
    # Loop through specified columns
    for col in columns:
        print('Working on column: {}'.format(col))
        
        # Set bounds of lower and upper 0.1%
        lower, upper = np.percentile(df[col], [0.01, 99.99])

        print('Upper bound: {}, Lower Bound: {}'.format(upper, lower))
        
        # Print the number of outliers
        print('Number of values labelled: ', len(df.loc[(df[col] > upper) | (df[col] < lower)]))
        
        # Set outlier column to true if outlier
        df = df.loc[(df[col] <= upper) & (df[col] >= lower)]
        
    return df

In [ ]:
# Run function to label outliers and assign to cars dataframe
cars = remove_outliers(cars, ['year', 'price'])

In [ ]:
cars['year'].describe()

In [ ]:
# Drop natural gas observation
cars = cars.drop(cars.loc[cars['fuel'] == 'Natural Gas'].index)

**Reg**

Reg provides very similar information to the age/year features, therefore to avoid multi-collinearity in the models, drop this feature.

In [ ]:
cars = cars.drop(columns=['reg'])

## Transformations

In [ ]:
# Create 'new' column to consist of True and False (1 and 0)
cars['new'] = cars['condition'] == 'NEW'

# Get counts of new and used
cars['new'].value_counts()

In [ ]:
# Convert year to age
cars['age'] = 2022 - cars['year']

In [ ]:
# Log transformation of age.
cars['age_'] = np.log10(cars['age'])

In [ ]:
# Transform price using log base 10
cars['price_'] = np.log10(cars['price'])

In [ ]:
fig = plt.figure(figsize=(8,6))
cars['price_'].hist();

In [ ]:
# Transform mileage using square root
cars['mileage_'] = np.sqrt(cars['mileage'])

In [ ]:
sns.pairplot(data=cars.sample(10000,random_state=0)[['price_','age','mileage_']]);

In [ ]:
# Split mileage data into 5 equally spaced bins.
mileage_labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']
cars['mileage_bins'] = pd.qcut(cars['mileage'], q=[0, 0.2, 0.4, 0.6, 0.8, 1], labels=mileage_labels)

In [ ]:
# Group years into bins
year_labels = ['Very Old', 'Old', 'Neutral Age', 'New', 'Very New']
cars['year_bins'] = pd.qcut(cars['year'], q=[0, 0.2, 0.4, 0.6, 0.8, 1], labels=year_labels)

In [ ]:
# Create subplots for visualisation
fig, axs = plt.subplots(1, 2, figsize=(12,6), constrained_layout=True)

# Create box-plots of year_bins and mileage_bins against price
sns.boxplot(x='year_bins', y='price', data=cars, ax=axs[0], showfliers=False);
sns.boxplot(x='mileage_bins', y='price', data=cars, ax=axs[1], showfliers=False);

plt.savefig('figure/figure16.png')

### New Correlation and Predictive Power Score

In [ ]:
plt.figure(figsize=(10,8))

# Create annotated correlation heatmap
sns.heatmap(cars.corr(), annot=True, cmap="Blues", fmt='.2f');

In [ ]:
# Look at correlation for price and newly log transformed price.
print(np.abs(cars.corr()['price']).sort_values(ascending=False))
print('')
print(np.abs(cars.corr()['price_']).sort_values(ascending=False))

In [ ]:
# Build predictive power score table
pred_power = pps.predictors(cars.drop(columns=['price']), 'price_').sort_values(by='model_score')
pred_power

In [ ]:
# Build predictive power score table
pred_power = pps.predictors(cars.drop(columns=['price_']), 'price').sort_values(by='model_score')
pred_power

In [ ]:
pred_power = pps.predictors(cars.drop(columns='price'), 'price_').sort_values(by='model_score')
pred_power

In [ ]:
pred_power = pps.predictors(cars.drop(columns=['price','year','age_','year_bins','condition','mileage']), 'price_').sort_values(by='model_score')
pred_power

In [ ]:
# Create figure for plot
fig = plt.figure(figsize=(10,8))

# Barplot of predictive power score by feature
sns.barplot(x='ppscore', y='x', data=pred_power.loc[pred_power['model_score'] > 0], palette='Reds');
plt.xlabel('Predictive Power Score');
plt.ylabel('Feature');

plt.savefig('figure/figure5.png')

Feature Selection
- Age
- Model
- Make
- Type
- New
- Fuel
- Mileage_
- Colour
- Car_van

# Peparing for Machine Learning Models

## Train / Test Split

In [ ]:
# Create split for selected features
features = ['age','model','make','type','new','fuel','mileage_','colour','car_van']
X = cars[features]
y = cars['price_']

In [ ]:
# Create a train/test split for model building and validation
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.25, random_state=0
)

In [ ]:
print('Shape of training data: {}'.format(X_train.shape))
print('Shape of test data: {}'.format(X_test.shape))

X_train.head()

In [ ]:
# Take sample of the dataset for faster search times
sample = cars.sample(40000, random_state=0)
X_sample = sample[features]
y_sample = sample['price_']

# Create a sample data train/test split for model building and evaluation
SX_train, SX_test, Sy_train, Sy_test = train_test_split(
    X_sample, y_sample, 
    test_size = 0.25, random_state=0
)

In [ ]:
print('Shape of sample training data: {}'.format(SX_train.shape))
print('Shape of sample test data: {}'.format(SX_test.shape))

## Baseline Models

### Recording Baseline Results

In [ ]:
# Import library for recording evaluation metrics
from prettytable import PrettyTable
table = PrettyTable()
table.field_names = ['Model', 'MAE', 'MAPE', 'RMSE', 'R2-Score']
table.sortby = "MAE"

In [ ]:
# Define function for transforming price_ back to price
def add_transformed_row(reg, X, y, label, table):
    y_pred = 10 ** reg.predict(X)
    y_true = 10 ** y
    
    mae = np.round(mean_absolute_error(y_pred, y_true),3)
    mape = np.round(mean_absolute_percentage_error(y_pred, y_true),3)
    rmse = np.round(mean_squared_error(y_pred, y_true, squared=False),3)
    r2 = np.round(r2_score(y_pred, y_true),3)
    
    table.add_row([label, mae, mape, rmse, r2])
    
    return y_pred, y_true

In [ ]:
# Dummy regressor to predict mean only
dummy = DummyRegressor(strategy='mean')
dummy.fit(SX_train, Sy_train)

y_pred, y_true = add_transformed_row(dummy, SX_test, Sy_test, 'Dummy Mean', table)
table

In [ ]:
# Dummy regressor to predict median only
dummy_med = DummyRegressor(strategy='median')
dummy_med.fit(SX_train, Sy_train)

add_transformed_row(dummy_med, SX_test, Sy_test, 'Dummy Median', table)
table

In [ ]:
# Simple linear regression model using mileage and age
reg = LinearRegression()

reg.fit(SX_train[['mileage_','age']], Sy_train)
y_pred, y_true = add_transformed_row(reg, SX_test[['mileage_','age']], Sy_test, 'Linear Regression Baseline', table)
table

In [ ]:
# Visualise the predicted/actual values of price
ax = sns.scatterplot(x=y_true, y=y_pred, alpha=0.5)
ax.set_xlabel('Actual Target Value')
ax.set_ylabel('Predicted Target Value')
ax.set_xlim(0, 300000)
ax.set_ylim(0, 300000)
ax.plot((0, 1000000), (0, 1000000), ':k', alpha=0.3, lw=1);

In [ ]:
# Show table of evaluation for basic models.
table

## Pipelines and Transformers

### One Hot Encoding

In [ ]:
# Show number of unique values in each feature
for feature in X.columns:
    print(feature, len(cars[feature].unique()))

In [ ]:
# Define features for one hot encoding
onehot_features = ['type','fuel']

# Build transformer for one hot encoding
onehot_transformer = Pipeline(
    steps=[ 
        ("ohe", OneHotEncoder(handle_unknown="ignore"))
    ]
)

# Fit the sample training data
onehot_transformer.fit(SX_train[onehot_features])

In [ ]:
# Trial the transformer
onehot = onehot_transformer.fit_transform(SX_train[onehot_features])
print('The shape of the array: ',onehot.todense().shape)
onehot.todense()

### Target Encoding

In [ ]:
# Define features for target encoding
target_features = ['make','model','colour']

# Build transformer for target encoding
target_transformer = Pipeline(
    steps=[
        ("tar", TargetEncoder()),
        ("scaler", StandardScaler())
    ]
)

# Fit the sample training data
target_transformer.fit(SX_train[target_features], Sy_train)

In [ ]:
target = target_transformer.fit_transform(SX_train[target_features], Sy_train)
print('The shape of the array: ', target.shape)
target

### Numeric Encoding

In [ ]:
# Define numeric features for scaling
num_features = ['mileage_','age']

# Building transformer for numeric features
num_transformer = Pipeline(
    steps=[
        ("scaler", StandardScaler())
    ]
)

# Fit the sample training data
num_transformer.fit(SX_train[num_features])

In [ ]:
# Check all is working correctly
num = num_transformer.fit_transform(SX_train[num_features])
print('The shape of the array: ', num.shape)
num

### Preprocessor Transformer

In [ ]:
# Build a preprocessor to run numeric, one hot and target transformers
preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_transformer, num_features),
        ("onehot", onehot_transformer, onehot_features),
        ("target", target_transformer, target_features),
    ],
    remainder='passthrough'
)

In [ ]:
# Trial the preprocessor
pre = preprocessor.fit_transform(SX_train, Sy_train)
print('Shape of the array: ', pre.shape)
pre

In [ ]:
# Define function pipeline to make building Pipelines easier
def pipeline(reg):
    pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("regressor", reg)])
    return pipeline

In [ ]:
features

In [ ]:
cols = num_features + onehot_features + target_features + ['new']
cols

## Building Intitial Models

### Linear Regression

In [ ]:
# Build linear regression model
reg = pipeline(LinearRegression())
reg.fit(SX_train, Sy_train)

In [ ]:
# View coefficients of linear model
reg[1].coef_

In [ ]:
# Get feature names from one hot transformer
reg['preprocessor'].transformers[1][1].get_feature_names_out(onehot_features)

In [ ]:
# Get feature names of numeric transformer
num_feat = list(reg['preprocessor'].transformers[0][1].get_feature_names_out(num_features))

# Get feature names of one hot transformer
one_hot_feat = list(reg['preprocessor'].transformers[1][1].get_feature_names_out(onehot_features))

# Build full feature names to replace columns
encoded_features = num_feat + one_hot_feat + target_features + ['new', 'car_van']

In [ ]:
# See how linear model ranks the features.
feat_imp = pd.DataFrame(reg[1].coef_)
feat_imp['name'] = encoded_features
feat_imp = feat_imp.sort_values(by=0, ascending=False)

# Plot feature importance coefficients
sns.barplot(x=0, y='name', data=feat_imp);

In [ ]:
# Add linear regression model to evaluation table
y_pred, y_true = add_transformed_row(reg, SX_test, Sy_test, 'Linear Regression (1)', table)
table

### Decision Tree Regressor

In [ ]:
# Build decision tree model
dtree = pipeline(DecisionTreeRegressor())
dtree.fit(SX_train, Sy_train)

In [ ]:
# Add decision tree model to evaluation table
y_pred, y_true = add_transformed_row(dtree, SX_test, Sy_test, 'Decision Tree (1)', table)
table

In [ ]:
# View MAE for decision tree with "price_" as reference
mean_absolute_error(dtree.predict(SX_test), Sy_test)

In [ ]:
# Get feature importances from decision tree
feat_imp = pd.DataFrame(dtree[1].feature_importances_)
feat_imp['name'] = encoded_features
feat_imp = feat_imp.sort_values(by=0, ascending=False)

# Visualise the feature importance
sns.barplot(x=0,y='name', data=feat_imp.head(12));

In [ ]:
# View the predicted/actual plot for decision tree model
ax = sns.scatterplot(x=y_true, y=y_pred, alpha=0.5)
ax.set_xlabel('Actual Target Value')
ax.set_ylabel('Predicted Target Value')
ax.set_xlim(0, 300000)
ax.set_ylim(0, 300000)
ax.plot((0, 300000), (0, 300000), ':k', alpha=0.3, lw=1);

### Random Forest Regressor

In [ ]:
# Build random forest model
forest = pipeline(RandomForestRegressor())
forest.fit(SX_train, Sy_train)

In [ ]:
# Add random forest model to evaluation table
y_pred, y_true = add_transformed_row(forest, SX_test, Sy_test, 'Random Forest (1)', table)
table

In [ ]:
# View predicted/actual plot for random forest
ax = sns.scatterplot(x=y_true, y=y_pred, alpha=0.5)
ax.set_xlabel('Actual Target Value')
ax.set_ylabel('Predicted Target Value')
ax.set_xlim(0, 300000)
ax.set_ylim(0, 300000)
ax.plot((0, 300000), (0, 300000), ':k', alpha=0.3, lw=1);

In [ ]:
# Get feature importance from random forest
feat_imp = pd.DataFrame(forest[1].feature_importances_)
feat_imp['name'] = encoded_features
feat_imp = feat_imp.sort_values(by=0, ascending=False)

# Visualise feature importance
sns.barplot(x=0, y='name', data=feat_imp.head(12));

In [ ]:
# Evaluate MAE for reference
mean_absolute_error(forest.predict(SX_test), Sy_test)# 

### Ridge Regression

In [ ]:
# Build ridge regression model
ridge = pipeline(Ridge())
ridge.fit(SX_train, Sy_train)

In [ ]:
# Add ridge regression model to evaluation table
y_pred, y_true = add_transformed_row(ridge, SX_test, Sy_test, 'Ridge Regression (1)', table)
table

### Support Vector Machines

In [ ]:
# Build SVM regression model
svm = pipeline(SVR())
svm.fit(SX_train, Sy_train)

In [ ]:
# Add SVM model to evaluation table
y_pred, y_true = add_transformed_row(svm, SX_test, Sy_test, 'SVM Regression (1)', table)
table

### Light GBM Regressor

In [ ]:
# Build Light GBM regression model
lgbR = pipeline(lgb.LGBMRegressor())
lgbR.fit(SX_train, Sy_train)

In [ ]:
# Add LGBM Regression model to evaluation table
y_pred, y_true = add_transformed_row(lgbR, SX_test, Sy_test, 'LGB Regressor', table)
table

### XGBoost Regressor

In [ ]:
# Build XGBoost regression model
xgbR = pipeline(xgb.XGBRegressor())
xgbR.fit(SX_train, Sy_train)

In [ ]:
# Visualise decision tree graph
xgb.plot_tree(xgbR[1], num_trees=0)
plt.rcParams['figure.figsize'] = [100, 20]
plt.show()

In [ ]:
# Add XGBoost model to evaluation table
y_pred, y_true = add_transformed_row(xgbR, X_test, y_test, 'XGBoost', table)
table

## Overfit / Underfit Trade Off

In [ ]:
# Set variables
depth = range(2, 50, 2)
train_mae = []
test_mae = []

# Loop through max_depth and record train/test MAE
for dep in depth:
    dtree = pipeline(DecisionTreeRegressor(max_depth=dep))
    dtree.fit(SX_train, Sy_train)
    
    train_mae.append(mean_absolute_error(10**dtree.predict(SX_train), 10**Sy_train))
    test_mae.append(mean_absolute_error(10**dtree.predict(SX_test), 10**Sy_test))

In [ ]:
# Create dataframe from values
fit_data = pd.DataFrame([list(depth), train_mae, test_mae]).transpose()
fit_data.columns = ['depth','train_mae','test_mae']
fit_data.sort_values(by='test_mae')

In [ ]:
# Visualise the train/test evaluation metric MAE for overfitting
fig = plt.figure(figsize=(8,6))
sns.lineplot(y=train_mae, x=depth);
sns.lineplot(x=depth, y=test_mae);
plt.xticks(range(0,48,2))

plt.show()

Begins to overfit the data at approximately max_depth = 14.

In [ ]:
dtree = pipeline(DecisionTreeRegressor(max_depth=14))
dtree.fit(SX_train, Sy_train)

In [ ]:
# Record new decision tree model in evaluation table
y_pred, y_true = add_transformed_row(dtree, SX_test, Sy_test, 'Decision Tree Max Depth 14', table)
table

In [ ]:
# Set variables
depth = range(2, 50, 2)
train_mae = []
test_mae = []

# Loop through max_depth and record train/test MAE
for dep in depth:
    forest = pipeline(RandomForestRegressor(max_depth=dep, n_jobs=2))
    forest.fit(SX_train, Sy_train)
    
    train_mae.append(mean_absolute_error(10**forest.predict(SX_train), 10**Sy_train))
    test_mae.append(mean_absolute_error(10**forest.predict(SX_test), 10**Sy_test))

In [ ]:
# Create dataframe from values
fit_data = pd.DataFrame([list(depth), train_mae, test_mae]).transpose()
fit_data.columns = ['depth','train_mae','test_mae']
fit_data

In [ ]:
min(fit_data['test_mae'])

In [ ]:
# Visualise the train/test evaluation metric MAE for overfitting
fig = plt.figure(figsize=(8,6))
sns.lineplot(y=train_mae, x=depth);
sns.lineplot(x=depth, y=test_mae);
plt.xticks(range(0,48,2))

plt.show()

In [ ]:
forest = pipeline(RandomForestRegressor(max_depth=18))
forest.fit(SX_train, Sy_train)

In [ ]:
# Record new decision tree model in evaluation table
y_pred, y_true = add_transformed_row(forest, SX_test, Sy_test, 'Random Forest Max Depth 18', table)
table

# Grid Search

In [ ]:
# Create new table for the best models to be found during grid search
best = PrettyTable()
best.field_names = ['Model', 'MAE', 'MAPE', 'RMSE', 'R2-Score']
best.sortby = "MAE"

## Ridge Regression

In [ ]:
grid_values = {'alpha':range(1,1000)}
scorers = ['neg_mean_absolute_error','neg_mean_absolute_percentage_error',
           'neg_root_mean_squared_error','r2']

In [ ]:
ridge = pipeline(GridSearchCV(Ridge(), param_grid = grid_values, scoring=scorers, refit='neg_mean_absolute_error'))
ridge

In [ ]:
ridge.fit(SX_train, Sy_train)

In [ ]:
cols_grid = ['rank_test_neg_mean_absolute_error','param_alpha', 'mean_test_neg_mean_absolute_error',
        'mean_test_neg_mean_absolute_percentage_error', 'mean_test_neg_root_mean_squared_error', 'mean_test_r2']

results = pd.DataFrame(ridge[1].cv_results_)[cols_grid]
results.sort_values(by='rank_test_neg_mean_absolute_error').head(5)

In [ ]:
params = ridge[1].best_params_
ridge = pipeline(Ridge(alpha=params['alpha']))
ridge.fit(X_train, y_train)

In [ ]:
y_pred, y_true = add_transformed_row(ridge, X_test, y_test, 'Ridge', best)
best

## Decision Tree Grid Search

In [ ]:
grid_values = {'max_depth': range(10,110, 5), 'min_samples_leaf': range(4,11)}
scorers = ['neg_mean_absolute_error','neg_mean_absolute_percentage_error',
           'neg_root_mean_squared_error','r2']

In [ ]:
dtree = pipeline(GridSearchCV(DecisionTreeRegressor(), param_grid = grid_values, scoring=scorers, refit='neg_mean_absolute_error'))
dtree

In [ ]:
dtree.fit(SX_train, Sy_train)

In [ ]:
cols_grid = ['rank_test_neg_mean_absolute_error','param_max_depth','param_min_samples_leaf', 'mean_test_neg_mean_absolute_error',
        'mean_test_neg_mean_absolute_percentage_error', 'mean_test_neg_root_mean_squared_error', 'mean_test_r2']

results = pd.DataFrame(dtree[1].cv_results_)[cols_grid]
results.sort_values(by='rank_test_neg_mean_absolute_error').head(5)

In [ ]:
params=dtree[1].best_params_
dtree = pipeline(DecisionTreeRegressor(max_depth=params['max_depth'], min_samples_leaf=params['min_samples_leaf']))
dtree.fit(X_train, y_train)

In [ ]:
y_pred, y_true = add_transformed_row(dtree, X_test, y_test, 'Decision Tree GridSearch', best)
best

### Random Forest

In [ ]:
# Set parameter values for search
grid_values = {'n_estimators':[10,50,100,200], 'max_depth':range(5,30,5), 'min_samples_leaf': range(3,9)}

# Set evaluation metrics for results
scorers = ['neg_mean_absolute_error','neg_mean_absolute_percentage_error',
           'neg_root_mean_squared_error','r2']

In [ ]:
# Initiate pipeline for grid search
forest = pipeline(GridSearchCV(RandomForestRegressor(), param_grid = grid_values, scoring=scorers, refit='neg_mean_absolute_error', n_jobs=4))
forest

In [ ]:
# Fit the sample data
forest.fit(SX_train, Sy_train)

In [ ]:
# Set columns for retrieval of results
cols_grid = ['rank_test_neg_mean_absolute_error','param_max_depth','param_min_samples_leaf', 'param_n_estimators',
             'mean_test_neg_mean_absolute_error','mean_test_neg_mean_absolute_percentage_error', 
             'mean_test_neg_root_mean_squared_error', 'mean_test_r2']

# Print the resulting dataframe with best result first
results = pd.DataFrame(forest[1].cv_results_)[cols_grid]
results.sort_values(by='rank_test_neg_mean_absolute_error').head(5)

In [ ]:
# Apply best parameters to whole training data
params = forest[1].best_params_
forest = pipeline(RandomForestRegressor(max_depth=params['max_depth'], min_samples_leaf=params['min_samples_leaf'], n_estimators=params['n_estimators'], n_jobs=3))
forest.fit(X_train, y_train)

In [ ]:
# Add evaluation metrics to table
y_pred, y_true = add_transformed_row(forest, X_test, y_test, 'Random Forest GridSearch', best)
best

In [ ]:
ax = sns.scatterplot(x=y_true, y=y_pred, alpha=0.5)
ax.set_xlabel('Actual Target Value')
ax.set_ylabel('Predicted Target Value')
ax.set_xlim(0, 400000)
ax.set_ylim(0, 400000)
ax.plot((0, 1000000), (0, 1000000), ':k', alpha=1, lw=1);

## XGBoost Regressor

In [ ]:
# Set grid values for search
grid_values = {
    'max_depth': range (4, 20, 2),
    'n_estimators': range(60, 220, 40),
    'learning_rate': [0.1, 0.01, 0.05]
}
scorers = ['neg_mean_absolute_error','neg_mean_absolute_percentage_error',
           'neg_root_mean_squared_error','r2']

In [ ]:
# Build pipeline for grid search of XGB regressor
xgbR = pipeline(GridSearchCV(xgb.XGBRegressor(), param_grid = grid_values, scoring=scorers, refit='neg_mean_absolute_error', n_jobs=3))
xgbR

In [ ]:
# Fit the grid search using sample data
xgbR.fit(SX_train, Sy_train)

In [ ]:
cols_grid = ['rank_test_neg_mean_absolute_error','param_max_depth','param_n_estimators', 'param_learning_rate', 'mean_test_neg_mean_absolute_error',
             'mean_test_neg_mean_absolute_percentage_error', 'mean_test_neg_root_mean_squared_error', 'mean_test_r2']

results = pd.DataFrame(xgbR[1].cv_results_)[cols_grid]
results.sort_values(by='rank_test_neg_mean_absolute_error').head(5)

In [ ]:
params = xgbR[1].best_params_
xgbR = pipeline(xgb.XGBRegressor(max_depth=params['max_depth'], n_estimators=params['n_estimators'], learning_rate=params['learning_rate']))
xgbR.fit(X_train, y_train)

In [ ]:
add_transformed_row(xgbR, X_test, y_test, 'XGBoost Grid Best', best)
best

## Light GBM Regressor

In [ ]:
# Set values of parameters
grid_values = {
    'max_depth': range (-1, 30, 5),
    'num_leaves': range(21, 41, 5),
    'learning_rate': [0.1, 0.01],
    'n_estimators': [50, 100, 200],
}

# Define evaluation metrics
scorers = ['neg_mean_absolute_error','neg_mean_absolute_percentage_error',
           'neg_root_mean_squared_error','r2']

In [ ]:
lgbR = pipeline(GridSearchCV(lgb.LGBMRegressor(), param_grid = grid_values, scoring=scorers, refit='neg_mean_absolute_error', n_jobs=3))
lgbR

In [ ]:
lgbR.fit(SX_train, Sy_train)

In [ ]:
cols_grid = ['rank_test_neg_mean_absolute_error','param_max_depth','param_n_estimators', 'param_learning_rate', 'param_num_leaves',
             'mean_test_neg_mean_absolute_error', 'mean_test_neg_mean_absolute_percentage_error', 
             'mean_test_neg_root_mean_squared_error', 'mean_test_r2']

results = pd.DataFrame(lgbR[1].cv_results_)[cols_grid]
results.sort_values(by='rank_test_neg_mean_absolute_error').head(5)

In [ ]:
params = lgbR[1].best_params_
lgbR = pipeline(lgb.LGBMRegressor(max_depth=params['max_depth'], n_estimators=params['n_estimators'], learning_rate = params['learning_rate'], n_jobs=3))
lgbR.fit(X_train, y_train)

In [ ]:
add_transformed_row(lgbR, X_test, y_test, 'LGBM Regressor', best)
best

# Model Evaluation and Analysis

We have found throughout the grid search that XGBoost performs best on the sample train/test divide, however we now need to confirm the findings by performing cross-validation on the entire dataset.

## Final Evaluation Metrics for Grid Search

In [ ]:
# Get final evaluation metrics from grid search
best

## Cross-Validation

Perform cross-validation on the selected models.

(Apologies if there is warnings)

In [ ]:
valid = PrettyTable()
valid.field_names = ['Model','Validation Mean Test MAE']
valid.sortby = 'Validation Mean Test MAE'

In [ ]:
validate = cross_validate(dtree, X, y, scoring='neg_mean_absolute_error', n_jobs=2)
valid.add_row(['Decision Tree', np.abs(validate['test_score'].mean())])

In [ ]:
validate = cross_validate(forest, X, y, scoring='neg_mean_absolute_error', n_jobs=2)
valid.add_row(['Random Forest', np.abs(validate['test_score'].mean())])

In [ ]:
validate = cross_validate(xgbR, X, y, scoring='neg_mean_absolute_error', n_jobs=2)
valid.add_row(['XGBoost', np.abs(validate['test_score'].mean())])

In [ ]:
validate = cross_validate(lgbR, X, y, scoring='neg_mean_absolute_error', n_jobs=2)
valid.add_row(['Light GBM', np.abs(validate['test_score'].mean())])

In [ ]:
valid

### Cross-Validate Predict

In [ ]:
from sklearn.model_selection import cross_val_predict

In [ ]:
# Get predictions for full dataset
valid_pred = cross_val_predict(xgbR, X, y, n_jobs=-1)
valid_pred

In [ ]:
# Print evaluation metrics
y_pred, y_true = 10**valid_pred, 10**y
print('The cross-validation evaluation metrics for XGBoost')
print('MAE: ',mean_absolute_error(y_pred, y_true))
print('MAPE: ', mean_absolute_percentage_error(y_pred, y_true))
print('RMSE: ', mean_squared_error(y_pred, y_true, squared=False))
print('R2: ', r2_score(y_pred, y_true))

## Feature Importance

Explainability is an important aspect in business when using machine learning models. A lot of the time it can be difficult to explain complex models in advanced machine learning techniques. Most ML models have built in ways to retrieve the importance of the features on predicting the target variable.

### Naming Features

In [ ]:
# Fit the training data
onehot_transformer.fit(X_train[onehot_features])
target_transformer.fit(X_train[target_features], y_train)
num_transformer.fit(X_train[num_features])

In [ ]:
# Get feature names of numeric transformer
num_feat = list(reg['preprocessor'].transformers[0][1].get_feature_names_out(num_features))

# Get feature names of one hot transformer
one_hot_feat = list(reg['preprocessor'].transformers[1][1].get_feature_names_out(onehot_features))

# Build full feature names to replace columns
encoded_features = num_feat + one_hot_feat + target_features + ['new', 'car_van']

### XGBoost

In [ ]:
# See how model ranks the features.
feat_imp = pd.DataFrame(xgbR[1].feature_importances_)
feat_imp['name'] = encoded_features
feat_imp = feat_imp.sort_values(by=0, ascending=False)

feat_imp

In [ ]:
len(feat_imp)

In [ ]:
# Build figure
fig = plt.figure(figsize=(10,8))

# Plot feature importance coefficients
sns.barplot(x=0, y='name', data=feat_imp);
plt.xlabel('Feature Importance')
plt.ylabel('Feature')

plt.savefig('figure/figure7.png')

plt.show()

### Random Forest

In [ ]:
forest = pipeline(RandomForestRegressor(max_depth=20, min_samples_leaf=3, n_estimators=200, n_jobs=-1))

In [ ]:
forest.fit(X_train, y_train)

In [ ]:
# See how model ranks the features.
feat_imp = pd.DataFrame(forest[1].feature_importances_)
feat_imp['name'] = encoded_features
feat_imp = feat_imp.sort_values(by=0, ascending=False)

feat_imp

In [ ]:
# Build figure
fig = plt.figure(figsize=(10,8))

# Plot feature importance coefficients
sns.barplot(x=0, y='name', data=feat_imp);
plt.xlabel('Feature Importance')
plt.ylabel('Feature')

plt.show()

### Ridge

In [ ]:
# See how model ranks the features.
feat_imp = pd.DataFrame(ridge[1].coef_)
feat_imp['name'] = encoded_features
feat_imp = feat_imp.sort_values(by=0, ascending=False)
feat_imp['abs'] = np.abs(feat_imp[0])
feat_imp

In [ ]:
# Build figure
fig, axs = plt.subplots(2, 1, figsize=(10,18))

# Plot feature importance coefficients
sns.barplot(x=0, y='name', data=feat_imp, ax=axs[0]);
axs[0].set_xlabel('Feature Importance Coefficient')
axs[0].set_ylabel('Feature')

feat_imp = feat_imp.sort_values(by='abs', ascending=False)

sns.barplot(x='abs', y='name', data=feat_imp, ax=axs[1]);
axs[1].set_xlabel('Absolute Feature Importance Coefficient')
axs[1].set_ylabel('Feature')

plt.show()

## SHAP
Another interesting way to visualise what our algorithm is predicting is using the SHAP library. Let's look at the best model XGBoost and how it comes to it's final predictions using SHAP.

In [ ]:
# Assign XGBoost to model variable and create dataframe of 1000 samples
model = xgbR[1]
X = pd.DataFrame(xgbR[0].transform(X_test[0:1000]).todense())

# Define column names
X.columns = encoded_features

In [ ]:
# Build SHAP explainer using model
explainer = shap.TreeExplainer(model)

In [ ]:
X.head()

In [ ]:
# Get SHAP values for tree explainer
shap_values = explainer.shap_values(X);

In [ ]:
# Visualise SHAP summary plot
shap.initjs()
shap.summary_plot(shap_values, X, show=False);

In [ ]:
row_idx = 0

shap.initjs()

shap.force_plot(
    explainer.expected_value,
    shap_values[row_idx],
    X.iloc[row_idx],
)

## Instance Level Errors
Where is the algorithm going wrong? Can it be improved by noticing which specific vehicles or categories of vehicles with the highest errors?

In [ ]:
# Transform back from log base 10
y_pred, y_true = 10**xgbR.predict(X_test), 10**y_test

In [ ]:
# Initiate figure
plt.figure(figsize=(12,8))


# Plot true vs predicted values
ax = sns.scatterplot(x=y_true, y=y_pred, alpha=0.5)
ax.set_xlabel('Actual Target Value')
ax.set_ylabel('Predicted Target Value')
ax.set_xlim(0, 400000)
ax.set_ylim(0, 400000)
ax.plot((0, 400000), (0, 400000), ':k', alpha=1, lw=1);

plt.savefig('figure/figure10.png')

In [ ]:
preds = pd.DataFrame([y_pred, y_true]).transpose()
preds.head()

In [ ]:
preds['diff'] = np.abs(preds[0] - preds[1])

In [ ]:
preds.head()

In [ ]:
test_data = X_test.copy()

In [ ]:
# Add residuals back to original test dataframe
test_data['price'], test_data['pred'] = y_true, y_pred
test_data['diff'] = test_data['price'] - test_data['pred']
test_data['diff_abs'] = np.abs(test_data['diff'])
test_data

In [ ]:
# Initiate figure
plt.figure(figsize=(12,8))

# Plot true vs predicted values
ax = sns.scatterplot(x='price', y='pred', data=test_data.loc[test_data['diff_abs'] < 50000], alpha=0.2);
ax = sns.scatterplot(x='price', y='pred', data=test_data.loc[test_data['diff_abs'] >= 50000], marker='o');
ax.set_xlabel('Actual Target Value')
ax.set_ylabel('Predicted Target Value')
ax.set_xlim(0, 400000)
ax.set_ylim(0, 400000)
ax.plot((0, 400000), (0, 400000), ':k', alpha=1, lw=1);

plt.savefig('figure/figure10.png')

In [ ]:
sns.scatterplot(x='price', y='diff_abs', data=test_data, alpha=0.3);

In [ ]:
# Create dataframe of errors > 50,000
big_diff = test_data.loc[test_data['diff_abs'] > 50000]
big_diff.head(10)

In [ ]:
# Create countplot of makes with large number of errors > 50,000
big_diff['make'].value_counts().sort_values(ascending=False).plot(kind='barh');

plt.savefig('figure/figure11.png')

In [ ]:
big_diff.loc[big_diff['price'] < 50000].head(10)

In [ ]:
ferrari = test_data.loc[test_data['make'] == 'Ferrari']
ferrari.head()

In [ ]:
# Create dataframe of ferrari's from training data
ferrari_train = X_train.loc[X_train['make'] == 'Ferrari']
ferrari_train['price_'] = 10**y_train
ferrari_train.head()

In [ ]:
# View size, mean and standard deviation of most variant ferrari's
ferrari_train.groupby('model')[['age','price_']].agg(['size','mean','std']).sort_values(by=('price_','std'), ascending=False).head()

In [ ]:
# View size, mean and standard deviation of most variant ferrari's
ferrari_box = ferrari_train.groupby('model')[['age','price_']].agg(['size','mean','std']).sort_values(by=('price_','size'), ascending=False).head()

In [ ]:
# Box-plot of most commonly occuring Ferrari's in training data
sns.boxplot(x='price_', y='model', data=ferrari_train.loc[ferrari_train['model'].isin(ferrari_box.index)]);

plt.savefig('figure/figure12.png')

In [ ]:
porsche = test_data.loc[test_data['make'] == 'Porsche']
porsche.head()

In [ ]:
porsche.groupby('model')[['age','price', 'diff']].agg(['mean','std'])

**Porsche Problems**

It appears that the large range in price of Porsche is causing issues in the models being built. With large importance being placed on the models feature,  Porsche models have a large variance in age and price. Where other vehicle makers might change the names of models, Porsche may keep the same model names for longer periods, causing this variance in age and/or price.

In [ ]:
sns.scatterplot(x='age', y='price', data=porsche);

In [ ]:
sns.boxplot(x='diff', data=porsche);